In [13]:
import PyPDF2
import getpass
from langchain_anthropic import ChatAnthropic
from dotenv import load_dotenv, find_dotenv
import os

In [14]:
# load API keys
_ = load_dotenv(find_dotenv())
# print(os.environ.get("ANTHROPIC_API_KEY"))

llm = ChatAnthropic(
    model="claude-3-sonnet-20240229",
    temperature=0,
    max_tokens=1024,
    timeout=None,
    max_retries=2,
    # other params...
)

In [15]:
def extract_text_from_pdf(pdf_path):
    # Open the PDF file
    with open(pdf_path, 'rb') as file:
        # Create a PDF reader object
        pdf_reader = PyPDF2.PdfReader(file)

        # Get the number of pages in the PDF
        num_pages = len(pdf_reader.pages)

        # Initialize a variable to store the extracted text
        text = ""

        # Loop through each page and extract text
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()

    return text


In [52]:
def get_keywords_from_anthropic(resume_text, job_text):
    messages = [
    (
        "system",
        f"You are a recruiter reviewing a candidates job profile.",
    ),
    ("human", f"This is a  resume : {resume_text}. Extract the following information from the given resume in a json. [The first key should be 'Years of Experience' if it doesn't show any experience or if the candidate is a recent graduate give 0 give me answer as one number and nothing else, The second key is 'Current Position' if any else return Student here,  and the third key is a 'Key Skills' this is a comma seprated list of five most relevant keyskills from this person's resume to this job description {job_text}. I only want skills that are present in the resume and relevant to the skills in {job_text} and it would be ideal if we have one skill from resume that is relevant to a different skill in {job_text}. All skills should be present in the resume]")
    ]
    keywords = llm.invoke(messages)
    return keywords


In [17]:
pdf_path = 'dummy_resume.pdf'
extracted_text = extract_text_from_pdf(pdf_path)
extracted_text

'First Last\nAdm. No. 22JEXXXX ♂phoneXXX-XXX-XXXX\n/envel⌢peﬁrstlast@gmail.com /linkedinlinkedin.com/in/ﬁrstlast /githubgithub.com/ﬁrstlast\nEducation\nUniversity Name Expected May 20XX\nBachelor of Science in Computer Science (GPA: 4.00 / 4.00) City, State\n•Relevant Coursework: Data Structures and Algorithms (C++), Prob & Stat in CS (Python), Intro to CS II (C++), Linear\nAlgebra w/Computational Applications (Python)\nExperience\nCompany Name 1 Jan 20XX – May 20XX\nSoftware Engineer City, State\n•Implemented microservices architecture using Node.js and Express, improving API response time by 25% and\nreducing server load by 30%.\n•Led a cross-functional team in implementing a new feature using React and Redux, resulting in a 20% increase in\nuser engagement within the ﬁrst month.\n•Optimized MySQL database queries, reducing page load times by 15% and enhancing overall application\nperformance.\nProjects\nProject Name 1 |React.js, Angular, Vue.js, Django, Flask, Ruby on Rails\n•Led th

In [18]:
job_data = { "job_skills": "Java,SpringBoot,OOPS,AWS,SQL",
         "years_of_experience": "2",
         "Job_title" : "Java Backend Engineer"
        }
job_text = str(job_data)


In [53]:
keyword_from_resume = get_keywords_from_anthropic(extracted_text, job_text)

In [54]:
import json
json.loads(keyword_from_resume.content)

{'Years of Experience': 0,
 'Current Position': 'Student',
 'Key Skills': 'Java, Spring Boot, SQL, TypeScript, React.js'}